In [1]:
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import seaborn as sns
import os

from datetime import datetime

plt.style.use("ggplot")

# Get water chemistry data from NIVA's database

This notebook retrieves, filters, and exports water chemistry and station metadata from NIVA’s database using the `nivapy3` Python library. It is intended to be run in NIVA’s JupyterLab-based research environment.

### How to run the analysis?
1. Clone the AquaINFRA GitHub repo or copy the notebook to your environment in NIVA's JupyterHub.
2. Make sure the following dependencies are installed in the environment: pandas, nivapy3, matplotlib, seaborn, os
3. Login with Oracle username and password.
4. Define the project, stations, and parameters of interest.
5. Run all cells sequentially.

### Which programming languages are used?
- Python

### How does the workflow look like?
1. Database connection
2. Project selection by filtering project names
3. Station retrieval and filtering
4. Interactive map visualization of selected stations
5. Water chemistry parameter discovery for a given date range
6. Water chemistry data retrieval
7. Export of station metadata and water chemistry to CSV files

### Which functions are used and how are they connected?
- `nivapy.da.connect()` – Connects to the database
- `nivapy.da.select_ndb_projects()` – Fetches all projects
- `nivapy.da.select_ndb_project_stations()` – Retrieves stations for a selected project
- `nivapy.spatial.quickmap()` – Visualizes stations on a map
- `nivapy.da.select_ndb_station_parameters()` – Lists available chemistry parameters for given stations and time span
- `nivapy.da.select_ndb_water_chemistry()` – Fetches the water chemistry data

### Anything else needed to run the analysis?
- Access to the Oracle database and user credentials
- Oracle client libraries installed in the environment
- A working clone or path reference to the `nivapy3` library [https://github.com/NIVANorge/nivapy3](https://github.com/NIVANorge/nivapy3)

**Note**: For internal use only. External users may need adaptations due to restricted database access.

## 0. User Inputs

In [2]:
# Define project name
project_name = "Elveovervåkingsprogrammet"

# Define stations of interest
stations_to_keep = ['Drammenselva', 'Numedalslågen v/E18', 'Glomma, Sarpsfossen']

# Mapping full station names to simplified river names
river_mapper = {
    'Drammenselva': 'Drammenselva',
    'Numedalslågen v/E18': 'Numedalslagen',
    'Glomma, Sarpsfossen': 'Glomma'
}

# Period of interest
st_dt = "1980-01-01"
end_dt = "2024-08-31"

# Parameters of interest
par_list = ['NH4-N', 'NO3+NO2-N', 'NO3-N', 'TOTN_EF_usikker', 'SiO2', 'Si', 'TOC', 'TOTN', 
            'TOTP', 'TOTP_P', 'DOC', 'POC', 'STS', 'TSM', 'PO4-P', 'TOTP/F', 'UV-Abs. 410nm', 
            'Farge', 'UV-Abs. 254nm']

# Where to save data
base_dir = '../../data/river/water_chemistry/raw/' # Define base directory path
os.makedirs(base_dir, exist_ok=True) # Create directory (and parents) if it doesn't exist
today_str = datetime.today().strftime('%d-%m-%Y')  # For file naming

## 1. Login

In [3]:
# Login with Oracle username and password
eng = nivapy.da.connect()

Username:  ········
Password:  ········


Connection successful.


## 2. Get stations

In [4]:
# Get all projects in nivadatabase
proj_df = nivapy.da.select_ndb_projects(eng)
proj_df.head()

1536 projects in the NIVADATABASE.


,project_id,project_name,project_description
0,1,xxx,None
1,55,PARCOM_Aquateam,None
2,56,PARCOM_NIVA,None
3,58,MAR_BOK,Fagdata
4,59,MAR_IMA,Fagdata


In [5]:
# Filter to just the project 
proj_df = proj_df.query(
    "project_name.str.contains(@project_name)", engine="python"
)
proj_df

,project_id,project_name,project_description
995,10306,Elveovervåkingsprogrammet,Etterfølgeren til RID. Overvåking av tilførsle...


In [6]:
stn_df = nivapy.da.select_ndb_stations(eng)
stn_df.head()

30503 stations in the NIVADATABASE.


,station_id,station_code,station_name,station_type,latitude,longitude
0,3561,BUSEDRA,Drammenselva,Elv,59.753992,10.009029
1,3562,FINEALT,Altaelva.,Elv,69.901142,23.287058
2,3563,NOREVEF,Vefsna.,Elv,65.749464,13.239104
3,3564,ROGEORR,Orreelva,Elv,58.731430,5.529362
4,3565,ROGESUL,Suldalslågen.,Elv,59.482215,6.259855


In [7]:
# Select stations for the specific project
stn_df = nivapy.da.select_ndb_project_stations(proj_df, eng, drop_dups=True)
stn_df.head()

,station_id,station_code,station_name,station_type,longitude,latitude
0,40352,BUSEDRA,Drammenselva,Elv,10.007898,59.752576
1,40353,SKI,Skienselva,Elv,9.606396,59.203230
2,40354,OTR2,Otra v/Skråstad,Elv,7.958165,58.184772
3,40355,NUM1,Numedalslågen v/E18,Elv,10.070949,59.101283
4,40356,CGL SAR,"Glomma, Sarpsfossen",Elv,11.134112,59.279802


In [8]:
# Show stations
nivapy.spatial.quickmap(
    stn_df, cluster=True, aerial_imagery=True, kartverket=True, popup="station_code"
)

In [9]:
# Select the stations of interest
sel_stn_df = stn_df[stn_df['station_name'].isin(stations_to_keep)]

# Show stations
nivapy.spatial.quickmap(
    sel_stn_df, cluster=True, aerial_imagery=True, kartverket=True, popup="station_code"
)

In [10]:
# Save metadata to csv 
for full_station_name, simple_river_name in river_mapper.items():
    river_meta_df = sel_stn_df[sel_stn_df['station_name'] == full_station_name]
    
    if not river_meta_df.empty:
        filename = f'water_chem_{simple_river_name}_metadata_{today_str}.csv'
        filepath = os.path.join(base_dir, filename)
        river_meta_df.to_csv(filepath, index=False)
        print(f'Saved {filepath}')
    else:
        print(f'No data for station: {full_station_name}')

Saved ../../data/river/water_chemistry/raw/water_chem_Drammenselva_metadata_11-06-2025.csv
Saved ../../data/river/water_chemistry/raw/water_chem_Numedalslagen_metadata_11-06-2025.csv
Saved ../../data/river/water_chemistry/raw/water_chem_Glomma_metadata_11-06-2025.csv


## 3. Get parameters of interest 

In [11]:
# Find water chemsitry parameters available for a set of stations within a period of interest
par_df = nivapy.da.select_ndb_station_parameters(sel_stn_df, st_dt, end_dt, eng)

# Filter to parameters of interest
par_df = par_df.query("parameter_name in @par_list")
par_df

/opt/conda/lib/python3.11/site-packages/nivapy3/da.py:440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stn_df["station_id"].drop_duplicates(inplace=True)


123 parameters available for the selected stations and dates.


,parameter_id,parameter_name,unit
35,1539,DOC,mg/L C
36,570,DOC,mg/l
40,288,Farge,mg Pt/l
57,8,NH4-N,µg/l
58,1625,NO3+NO2-N,µg/l
59,7,NO3-N,µg/l
75,12,PO4-P,µg/l
76,572,POC,µg/l C
87,396,STS,mg/l
90,3297,Si,mg/l


## 4. Get water chemistry 

In [12]:
wc_df, dup_df = nivapy.da.select_ndb_water_chemistry(
    sel_stn_df, par_df, st_dt, end_dt, eng, lod_flags=False, drop_dups=True
)

wc_df['station_name'] = wc_df['station_name'].replace('Glomma v/Sarpefossen', 'Glomma, Sarpsfossen') # The station name for Glomma differs between the water chemistry data and the metadata. I rename it here for consistency and to simplify saving and further processing.
wc_df

/opt/conda/lib/python3.11/site-packages/nivapy3/da.py:729: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stn_df["station_id"].drop_duplicates(inplace=True)
/opt/conda/lib/python3.11/site-packages/nivapy3/da.py:870: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["parameter_name"].fillna("", inplace=True)
/opt/conda/lib/python3.11/site-package

,station_id,station_code,station_name,sample_date,depth1,depth2,DOC_mg/L C,DOC_mg/l,Farge_mg Pt/l,NH4-N_µg/l,...,Si_mg/l,TOC_mg/l,TOTN_EF_usikker_µg/l,TOTN_µg/l,TOTP/F_µg/l P,TOTP_P_µg/l P,TOTP_µg/l,TSM_mg/l,UV-Abs. 254nm_Abs/cm,UV-Abs. 410nm_Abs/cm
,,,,,,,,,,,,,,,,,,,,,
0,40352,DRA1,Drammenselva,1990-01-11 00:00:00,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,350.0,NaN,NaN,6.0,0.80,NaN,NaN
1,40352,DRA1,Drammenselva,1990-02-12 00:00:00,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,419.0,NaN,NaN,7.0,2.00,NaN,NaN
2,40352,DRA1,Drammenselva,1990-03-13 00:00:00,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,405.0,NaN,NaN,6.0,1.40,NaN,NaN
3,40352,DRA1,Drammenselva,1990-04-18 00:00:00,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,399.0,NaN,NaN,5.0,1.30,NaN,NaN
4,40352,DRA1,Drammenselva,1990-05-21 00:00:00,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,317.0,NaN,NaN,6.0,1.70,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,40356,GLO1,"Glomma, Sarpsfossen",2024-06-03 04:20:00,0,0,3.0,NaN,19.0,8.0,...,1.50,3.0,NaN,420.0,NaN,7.8,7.9,4.42,0.113160,0.009802
1628,40356,GLO1,"Glomma, Sarpsfossen",2024-06-12 15:15:00,0,0,3.0,NaN,17.0,13.0,...,1.39,3.0,NaN,480.0,NaN,9.0,11.0,1.00,0.109750,0.009882
1629,40356,GLO1,"Glomma, Sarpsfossen",2024-06-20 11:30:00,0,0,3.8,NaN,24.0,9.0,...,1.49,3.7,NaN,470.0,NaN,8.5,11.0,4.63,0.144246,0.013710


In [13]:
# Save water chemistry to csv
for full_station_name, simple_river_name in river_mapper.items():
    river_df = wc_df[wc_df['station_name'] == full_station_name]
    
    if not river_df.empty:
        filename = f'water_chem_{simple_river_name}_raw_{today_str}.csv'
        filepath = os.path.join(base_dir, filename)
        river_df.to_csv(filepath, index=False)
        print(f'Saved {filepath}')
    else:
        print(f'No data for station: {full_station_name}')

Saved ../../data/river/water_chemistry/raw/water_chem_Drammenselva_raw_11-06-2025.csv
Saved ../../data/river/water_chemistry/raw/water_chem_Numedalslagen_raw_11-06-2025.csv
Saved ../../data/river/water_chemistry/raw/water_chem_Glomma_raw_11-06-2025.csv
